# Introduction

This notebook shows how to excecute the rolling horizon function from the <code>POSYDON.py</code> library, using your forecasts and real data for the wind speed, wave height and electricity prices. This notebook was used to produced the results in the paper.

In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import pickle
from POSYDON import *

**Import weather dataset**

In [2]:
weather_data = pd.read_csv("weather_ocean_data_new.csv", index_col=[0], parse_dates=True)
weather_data.head(2)

,NYE05.wind_speed_100m,NYE05.wave_height,RUWRF.wind_speed_100m,RUWRF.wind_dir_100m,NOAA.wave_height,NOAA.wave_dir,NOAA.wind_speed_10m,NOAA.wind_dir_10m
time,,,,,,,,
2019-08-12 00:00:00+00:00,7.993467,0.52,6.988079,234.920095,0.78,127.98,7.37,228.14
2019-08-12 01:00:00+00:00,7.987917,0.58,8.442663,233.856165,0.79,127.92,7.43,229.47


Select the relevant weather data (forecasts and true data for wind speed and wave height)

In [3]:
data = np.array(weather_data)
wind_speed_single_true = data[:,0]
wind_speed_single_NWP = data[:,2]
wave_height_true = data[:,1]
wave_height_NWP = data[:,4]

**Import and process day-ahead electricity price dataset**

In [4]:
PJM_price_data = pd.read_csv('True_n_LEAR_el_prices_da_comed.csv', index_col=0, parse_dates=True) # Load PJM dataset

el_prices_real = np.array(PJM_price_data['Real price']).reshape(-1)
el_prices_real += 24 #incentive

el_prices_forecast = np.array(PJM_price_data['LEAR 56']).reshape(-1)
el_prices_forecast += 24 #incentive

**Specify other optimization parameters and prepare to run optimization model**

In [5]:
Din = 15 # starting day from beginning of dataset (cannot be 0 because historical data are needed to generate scenarios)
N_t=5 # number of wind turbines
N_S = 10 # number of scenarios of the stochastic optimization model
N_D = 9 # LTH length (the total length of the optimization horizon is N_D+1)
rel_gap = 0.001 # control relative optimality gap
exp_num = 1 # select which instance of the RUL experiment sets to solve
max_iter = 3 # number of horizon rolls. If None, then the model is solved until it reaches the end of the dataset

# Bring wind speed data to appropriate shape (time_dim, N_t)
wind_speed_true = np.repeat(wind_speed_single_true[:,np.newaxis],N_t,1)
wind_speed_NWP = np.repeat(wind_speed_single_NWP[:,np.newaxis],N_t,1)

# Create copies of dataset before importing them to the rolling horizon function
wind_speed_true=wind_speed_true.copy()
wave_height_true=wave_height_true.copy()
wind_speed_forecast=wind_speed_NWP.copy()
wave_height_forecast=wave_height_NWP.copy()
el_prices_true=el_prices_real.copy()
el_prices_forecast=el_prices_forecast.copy()

**Solve the model with a rolling horizon algorithm**

In [7]:
# Solve the model by the rolling horizon algorithm, and store solutions to sol
sol = solveRollingHorizon( wind_speed_true,
                wave_height_true,
                wind_speed_forecast,
                wave_height_forecast,
                N_S,
                Din,
                el_prices_true,
                el_prices_forecast,
                degr_experiment_num=exp_num,
                deg_data_folder="Data/Degradation Signal info/Data for Optimization",
                power_curve_folder="Data/Yaw power curve",
                mip_gap=rel_gap,
                N_D=N_D,
                max_iter=max_iter)

# Save the solutions in a json file
sol_file = open("test.json", "wb")
pickle.dump(sol, sol_file)
sol_file.close()

Data prepared in  1.4052  sec
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 714717
Academic license - for non-commercial use only - registered to peterpap94@gmail.com
Discarded solution information
##########################################################
# POSYDON: Yaw misalignment and maintenance optimization #
##########################################################
Set parameter MIPGap to value 0.001
Set parameter TimeLimit to value 1800
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to peterpap94@gmail.com
Optimize a model with 21411 rows, 16356 columns and 123847 nonzeros
Model fingerprint: 0x73298554
Variable types: 6930 continuous, 9426 integer (9256 binary)
Coefficient statistics:
  Matrix range 

Found heuristic solution: objective 311879.22716

Root relaxation: objective 3.154601e+05, 1199 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    315460.14925 315460.149  0.00%     -    0s

Explored 1 nodes (1199 simplex iterations) in 0.68 seconds (0.39 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 315460 311879 311233 

Optimal solution found (tolerance 1.00e-03)
Best objective 3.154601492539e+05, best bound 3.154601492539e+05, gap 0.0000%
Data prepared in  0.0  sec
Discarded solution information
*** Initiating day-ahead simulation from input schedule ***
Set parameter TimeLimit to value 1800
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8

In [8]:
sol.keys()

dict_keys(['stochastic_sol', 'stochastic_input', 'simulation', 'true_input', 'time_per_roll', 'equivalent_times'])

In [12]:
sol['equivalent_times'] #shows how many equivalent days of nominal loading the wind turbine has worked

array([[42.        , 53.        , 29.        , 36.        , 23.        ],
       [42.36907583, 53.34916222, 29.37325782, 36.36907583, 23.36923852],
       [42.87030849, 53.83683596, 29.85209348, 36.80674061, 23.84416203],
       [43.67724136, 54.64962941, 30.64568277, 37.65857019, 24.70476565]])

In [11]:
sol['stochastic_sol'][0].keys()

dict_keys(['STH_sched', 'STH_yaw', 'LTH_sched', 'LTH_yaw', 'DMC', 'crit_coef', 'lambda_is', 'expected_STH_cost', 'remaining_maint_hrs', 'other_params', 'model'])